# Statistical tests:
* The goal here is to use statistical tests to identify or confirm correlations and common patterns.  Correlation tests include the correlation coefficient (sometimes known as R2) and cross-correlation. Correlation coefficients test if variables shift value in sync.  Cross-correlation compares pairs of time series and tests for consistent long-term patterns.  We will also look for periodicities. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d
import pandas as pd
from datetime import datetime
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
import datetime
def dateparse (date_string):
    return datetime.datetime.strptime(date_string, '%d-%m-%Y %H:%M:%S')

## Step One - Gather Data

Plume Bending Data

In [ ]:
# Test path to data using unix command 
#!ls /home/jovyan/data/covis_data/

## notes:
1. For direction, units are degrees. Both 0 degrees and 360 degrees are North.  Probably Magnetic North rather than True North.
2. Angle from vertical is also in degrees.
3. The time is in a modified Julian Day format. Modified in that it is only days since January of the appropriate year AND that it likely starts at midnight not noon.
4. The BendData are a linear fit to a centerline of an isosurface  representing a plume.

In [ ]:
path = 'covis/diaz/Covis_Data/angles_partialpts_2010.dat' # use your path
df_bd2010 = pd.read_csv(path, sep=",")
df_bd2010['year'] = '2010'
df_bd2010['datetime'] = pd.to_datetime(df_bd2010.year, format='%Y') + pd.to_timedelta(df_bd2010.jday - 1, unit='d')
df_bd2010['datetime'] = df_bd2010['datetime'].dt.round('1s')
df_bd2010 = df_bd2010.set_index('datetime')
df_bd2010.drop(['jday', 'year'], axis=1,inplace=True)
df_bd2010 = df_bd2010.rename_axis(None)
df_bd2010 = df_bd2010.resample('h').mean()
df_bd2010.head(25)

In [ ]:
#df_bd2010.hvplot(x = 'index', y= ['direction'], kind = 'scatter') * df_bd2010.hvplot(x = 'index', y= ['direction'])

In [ ]:
#df_bd2010.hvplot(x = 'index', y= ['direction', 'deg_from_vert'])

## Weather data
* I'm not sure how useful this file will be or what timeframe it covers.  An explanation of the variable names can be found at https://www.ndbc.noaa.gov/measdes.shtml
* weather_data_for_plotting*.txt files
* 13 variables (Year, Month, Day, Hour, Minute, Seconds, Julian day, wave height, wind direction, wind speed, wind gust speed, atmospheric pressure, air temperature)
* separator = single blank space
### notes:
1. Time is given both as a vector (from original data file probably) and as a Julian day.
2. No information on other units by m/s likely for speeds.  Rest should match the NOAA information.  
3. I probably still have the original NOAA data files for these buoys. I'll see if I can match data files to the time periods; my directories seemed a little confused.

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv' # use your path
df_wd2010c46036 = pd.read_csv(path, sep=",")
df_wd2010c46036['year']= df_wd2010c46036['year'].astype(int).astype(str)
df_wd2010c46036['month']= df_wd2010c46036['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['day']= df_wd2010c46036['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['hour']= df_wd2010c46036['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['minute']=df_wd2010c46036['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['datetime'] = df_wd2010c46036['year'] + df_wd2010c46036['month'] + df_wd2010c46036['day'] +\
'T' + df_wd2010c46036['hour']+ ':' + df_wd2010c46036['minute']
df_wd2010c46036['datetime'] = pd.to_datetime(df_wd2010c46036['datetime'])
df_wd2010c46036 = df_wd2010c46036.set_index('datetime')
df_wd2010c46036 = df_wd2010c46036[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2010c46036 = df_wd2010c46036.rename_axis(None)
df_wd2010c46036 = df_wd2010c46036.resample('h').mean()
df_wd2010c46036.head(10)

In [ ]:
test = pd.merge(df_wd2010c46036, df_bd2010,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis = pd.DataFrame()
df_covis = test[test['_merge'] == 'both']
del df_covis['_merge']
df_covis = df_covis.dropna()
df_covis.head()

In [ ]:
df_covis.describe()

In [ ]:
!ls /home/jovyan/data/covis_data/recomputed/

In [ ]:
# Take quick look at data using unix command 
!echo 'date,inclination,azimuth' | cat - ~jovyan/data/covis_data/recomputed/angles_partialpts_2010.dat > ~jovyan/data/covis_data/temp.txt
!sed 's/ /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv
!head /home/jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv

In [ ]:
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv' # use your path
df_rc2010 = pd.read_csv(path, sep=",")
df_rc2010['year'] = '2010'
df_rc2010['datetime'] = pd.to_datetime(df_rc2010.year, format='%Y') + pd.to_timedelta(df_rc2010.date - 1, unit='d')
df_rc2010['datetime'] = df_rc2010['datetime'].dt.round('1s')
df_rc2010 = df_rc2010.set_index('datetime')
df_rc2010.drop(['date', 'year'], axis=1,inplace=True)
df_rc2010 = df_rc2010.rename_axis(None)
df_rc2010 = df_rc2010.resample('h').mean()
df_rc2010.head(5)

In [ ]:
test = pd.merge(df_wd2010c46036, df_rc2010,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc = pd.DataFrame()
df_covis_rc = test[test['_merge'] == 'both']
del df_covis_rc['_merge']
df_covis_rc = df_covis_rc.dropna()
del df_covis_rc['jday']
df_covis_rc.head()

In [ ]:
weather = df_covis_rc.hvplot.scatter(y='wnd_dir', color='wnd_spd',
                                     cmap='colorwheel', s= 'wnd_spd',
                                     scale = 2, height=200,
                                     title= 'Example Plots',
                                     ylabel= 'Wind Direction (deg)',
                                     ylim = (0, 400),hover_cols=['wv_height'])
bending = df_covis_rc.hvplot.scatter(y='azimuth',color='inclination',
                                     cmap='viridis', s= 'wnd_spd',
                                     scale = 2, height=200,
                                     ylabel= 'Azimuth (deg)',
                                     xlabel = 'Time (h)',
                                     ylim = (-160, 150)).hist() 
(weather + bending).cols(1)

### Above we have two figures that allow for comparison over time of the various parameters we are interested. The top figure uses the y axis to plot wind direction, and uses both color and dot size to communicate wind speed. 

### The second plot uses the y axis to plot azimuth and uses color to communicate inclination... the wind speed is still communicated using size.

## Issues:
1. Both correlation coefficient estimation and cross-correlation computation assume that the values in the different datasets correspond … here it would be in time.  So we will need to do some interpolation to get our time series onto similar sampling times.
2. Most estimators of periodicity assume regular or uniform spacing of the data. There are, however, methods for working with data with gaps.
3. Periodicity estimates also change with the length of the data set.  So we will need to think about how to best compare the 2010 and 2011 data sets given the short 2010 record.

In [ ]:
df_covis_rc.hvplot.scatter(x = 'wnd_dir', y= 'azimuth')

In [ ]:
df_covis_rc.hvplot.scatter(x = 'wnd_spd', y= 'inclination')

In [ ]:
df_covis_rc.wnd_spd.hvplot.violin(by='index.day')

## A game plan for step 3:
1. Extract timing information for the various data sets, especially start time, stop time, and time intervals.  We need both the typical (not necessarily average) time step as well as some information on the number and size of data gaps.  We will use this to determine how to proceed with the correlation steps.  In particular, it will be useful to know which data are sampled at similar or faster rates than the plume bending data.
2. Estimate some basic statistics for key data sets: means, standard deviations, obvious trends.  This will be useful in defining what extreme bending means if nothing else.
3. Identify which data sets are key. Which data sets are central to the goals or questions?  Which data sets seem to have common features, patterns, or trends?  This may require plotting data together in ways or combinations that have not yet seemed obvious.
4. Interpolate data onto a common time sampling scheme.  All the COVIS data should be at similar but not necessarily identical times: the vertical velocity data will be offset about 20-40 minutes later than the bending data and may have more gaps.  I don't remember about the weather data and each buoy may be different.  I forget what else there is that I gave you.  I think there is some relevant data that we have not yet pulled from its repository.

So you might think a little about what information you'd like to have but don't.
Once interpolated to a common sampling, you can estimate the correlation coefficient.  This expects a linear pattern so it might or might not pick up on all relationships that exist.  Also, some of the directions in the bending data are not correct.  So I need to get you an updated data file soon. But you can figure out how  to do this with the existing data while I figure out the problems with the old data.
Cross-correlation is the next step after that.
Then we need to figure out about periodicities.  The Lomb-Scargle method is the best one for this data.  I think it exists in python but we will need to track that information down.
Another things to think about is the definition of a weather event.  What is a storm? How will it appear in the data? What does that mean about what we expect to find in the bending data.

What is a bending event? 
What is a storm? 